<a href="https://colab.research.google.com/github/Computer-CGuy/etherpaisahipaisa/blob/master/Ether_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --quiet cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8-cp37-cp37m-linux_x86_64.whl

     |████████████████████████████████| 144.6MB 64kB/s 
     |████████████████████████████████| 61kB 3.2MB/s 
ERROR: earthengine-api 0.1.260 has requirement google-api-python-client<2,>=1.12.1, but you'll have google-api-python-client 1.8.0 which is incompatible.


In [3]:
import torch_xla
import torch_xla.core.xla_model as xm

In [4]:
# CORES = len(torch_xla.core.xla_model.get_xla_supported_devices())
# print("Number of cores",CORES)

In [5]:
import torch
import time

In [6]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = xm.xla_device()
print("Using",str(device),"as your PyTorch Device")

Using xla:1 as your PyTorch Device


In [7]:
ROUNDS = 24
dtype = torch.long

def ROL64(a,offset):
    # p = torch.clone(a)
    # # print(offset[1].shape)
    # # print(p.shape)
    # p/=(offset[1])
    return ((a*offset[0]) ^ (a/offset[1]).long())

def tu_ROL64(a,offset):
    p = torch.clone(a)
    print(offset[:,1].shape)
    print(p.shape)
    p/=(offset[:,1])
    return ((a*offset[:,0]) ^ p)
def index(x,y):
    return (((x)%5)+5*((y)%5))

In [8]:
KeccakRoundConstants = torch.zeros((25), dtype=dtype).to(device)

one = torch.LongTensor([1,2]).to(device)
# KeccakRhoOffsets = torch.LongTensor([0 , 1 , 62, 28, 27, 36, 44, 6, 55, 20, 3, 10,43, 25, 39, 41, 45, 15, 21, 8, 18, 2, 61, 56, 14]).to(device)
KeccakRhoOffsets = torch.ones([2,25,1],dtype=dtype).to(device)

# For pi

shift_pi = torch.zeros([25,25],dtype=torch.int64).to(device)
l =[0,6,12,18,24,3,9,10,16,22,1,7,13,19,20,4,5,11,17,23,2,8,14,15,21]
for g in range(25):
	shift_pi[g][l[g]]=1
shift_pi= shift_pi.unsqueeze(2)

shift_pi5D = torch.zeros([5,5],dtype=torch.int64).to(device)
l =[0,1,2,3,4]
for g in range(5):
	shift_pi5D[g][l[g]]=1
shift_pi5D= shift_pi5D.unsqueeze(2)



In [9]:
def tensorRoll5D(t,shift):
    ret = (t*shift_pi5D)
    return torch.sum(ret,dim=1)
    # t = ret[0]
    # for x in range(1,25):
    #     t+=ret[x]
    # return t

In [10]:
def tensorRoll(t,shift):
    ret = (t*shift_pi)
    return torch.sum(ret,dim=1)
    t = ret[0]
    for x in range(1,25):
        t+=ret[x]
    return t

In [11]:
def theta(A,C,D):
    
    C[:] = 0
    for x in range(5):
      C^=A[5*x:(x+1)*5]
    
    #   for y in range(5):
    #       C[x] ^= A[index(x, y)]
    # torch.bitwise_xor(A[0::5]^A[1::5]^A[2::5]^A[3::5],A[4::5],out=C)
    D = ROL64(C, one)
    # print(D.shape)
    # print("C")
    # # print((D[(x+1)%5]^C[(x+4)%5]).shape)
    # # # print((A[5*x:5*(x+1)]^(D[(x+1)%5]^C[(x+4)%5])).shape)
    # # return 
    # print(C.shape)
    # print(D.shape)
    # print(A.shape)
    # D = torch.roll(D,1,dims=0)
    # C = torch.roll(C,4,dims=0)
    D = tensorRoll5D(D,1)
    C = tensorRoll5D(D,4)
    # A^=
    for x in range(5):
        A[5*x:5*(x+1)] = A[5*x:5*(x+1)]^D^C
        # for y in range(5):
        # #     # print(A[index(x, y)].shape)
        # #     # print(D[(x+1)%5].shape)
        # #     # print(C[(x+4)%5].shape)
        #     A[index(y, x)] = A[index(y, x)]^D[(x+1)%5] ^ C[(x+4)%5]
        # A[x::5] = A[x::5]^D[(x+1)%5] ^ C[(x+4)%5]

def rho(A,C,D):
    # for x in range(5):
    #   for y in range(5):
    A = ROL64(A, KeccakRhoOffsets)

def pi(A,C,D):
    # (A*shift_pi)
    # torch.sum((A*shift_pi),axis=1,out=A)
    # temp = torch.clone(A)
    # for x in range(5):
    #   for y in range(5):
    #       A[index(0*x+1*y, 2*x+3*y)] = temp[index(x, y)]
    # torch.tensordot(shift_pi,A,dims=1,out=A)
    ret = (A*shift_pi)
    # torch.sum
    return torch.sum(ret,dim=1)
    # A = ret[0]
    # for x in range(1,25):
    #     A+=ret[x]
def chi(A,C,D):
    for y in range(5):
        C = (A^tensorRoll(~A,1) & tensorRoll(A,2))[5*(y):5*y+5]
        # for x in range(5):
        #     C[x] = A[index(y, x)] ^ ((~A[index(y, x+1)]) & A[index(y, x+2)])
        # # for x in range(5):
        A[5*y:5*(y+1)] = C

def iota(A,indexRound):
    A[0] ^= KeccakRoundConstants[indexRound]

def mine(state,C,D):
    for i in range(ROUNDS):
        theta(state,C,D)
        rho(state,C,D)
        pi(state,C,D)
        chi(state,C,D)
        iota(state, i)
        # theta(state,C,D)
        # print("Theta",time.time()-t2)
        # t2 = time.time()
        # rho(state,C,D)
        # print("Rho",time.time()-t2)
        # t2 = time.time()
        # pi(state,C,D)
        # print("Pi",time.time()-t2)
        # t2 = time.time()
        # chi(state,C,D)
        # print("Chi",time.time()-t2)
        # t2 = time.time()
        # iota(state, i)
        # print("Iota",time.time()-t2)
        # t2 = time.time()


In [12]:
def mp_fn(index):
    global batch
    global t0
    state = torch.zeros([25,batch],dtype=dtype).to(device)
    C = torch.zeros([5,batch],dtype=dtype).to(device)
    # C = torch.zeros([25,batch],dtype=dtype).to(device)
    D = torch.clone(C)
    # D = torch.zeros([25,batch],dtype=dtype).to(device)
    t0= time.time()
    mine(state, C, D)
    print(state.shape)
batch = 3000000
t0 = time.time()

print(f"{batch:,}")
mp_fn(0)

t1 = time.time() - t0
t1/=batch
print(f"{1/t1:,}")

3,000,000
torch.Size([25, 3000000])
35,432,246.58290297


In [40]:
import  torch_xla.debug.metrics as met

print(met.metrics_report())

Metric: CompileTime
  TotalSamples: 52
  Accumulator: 02s318ms172.436us
  ValueRate: 910.103us / second
  Rate: 0.0204149 / second
  Percentiles: 1%=008ms499.100us; 5%=009ms632.799us; 10%=009ms032.350us; 20%=010ms016.534us; 50%=012ms772.237us; 80%=013ms701.875us; 90%=014ms870.028us; 95%=017ms231.120us; 99%=02s724ms227.384us
Metric: DeviceLockWait
  TotalSamples: 232
  Accumulator: 810.743us
  ValueRate: 000.318us / second
  Rate: 0.0910208 / second
  Percentiles: 1%=001.270us; 5%=001.400us; 10%=001.565us; 20%=001.786us; 50%=003.043us; 80%=003.800us; 90%=004.588us; 95%=005.363us; 99%=010.390us
Metric: ExecuteTime
  TotalSamples: 232
  Accumulator: 695ms937.492us
  ValueRate: 272.830us / second
  Rate: 0.0910824 / second
  Percentiles: 1%=002ms838.040us; 5%=002ms020.643us; 10%=002ms108.972us; 20%=002ms231.355us; 50%=003ms590.696us; 80%=003ms977.432us; 90%=003ms207.214us; 95%=004ms940.692us; 99%=012ms878.734us
Metric: InboundData
  TotalSamples: 232
  Accumulator: 2.42KB
  ValueRate: 0.97